In [26]:
import pickle
import numpy as np

with open('drive/MyDrive/X.pickle', 'rb') as file:
    X = pickle.load(file)

with open('drive/MyDrive/Y.pickle', 'rb') as file:
    Y = pickle.load(file)

x_train = np.array(X)
y_train = np.array(Y)

In [27]:
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential

# fraction of the input to drop; helps prevent overfitting
seq_len = 25
dropout = 0.5
window_size = seq_len // 2

# Adjust input shape
input_shape = (1, 86)
num_classes =  15

model = Sequential()

model.add(LSTM(window_size, return_sequences=True, input_shape=input_shape))
model.add(Dropout(rate=dropout))
model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True)))
model.add(Dropout(rate=dropout))
model.add(Bidirectional(LSTM(window_size, return_sequences=False)))
model.add(Dense(units=num_classes, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))


In [28]:
import pickle
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

batch_size = 64
y_train = tf.cast(y_train, tf.int32)
y_train = tf.one_hot(y_train, num_classes)
optimizer = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer , metrics=['accuracy'])

total = 0
step = 500
for r in range(0, 57500, step):
  print(total, r)
  with open('drive/MyDrive/trainable_embedding/te_'+str(r)+'_'+str((r+step))+'.pickle', 'rb') as file:
      X = pickle.load(file)
  l = len(X)
  x_train = np.array(X)
  x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))

  history = model.fit(
    x_train,
    y_train[total:total+l],
    epochs=10,
    batch_size=batch_size,
    shuffle=False,
    validation_split=0.1
  )

  total += l


0 0
Epoch 1/10
1088/1088 [==============================] - 23s 11ms/step - loss: 1.7610 - accuracy: 0.4455 - val_loss: 1.5084 - val_accuracy: 0.4954
Epoch 2/10
1088/1088 [==============================] - 9s 9ms/step - loss: 1.4952 - accuracy: 0.4913 - val_loss: 1.4388 - val_accuracy: 0.4953
Epoch 3/10
1088/1088 [==============================] - 9s 9ms/step - loss: 1.4555 - accuracy: 0.4957 - val_loss: 1.4137 - val_accuracy: 0.5095
Epoch 4/10
1088/1088 [==============================] - 10s 9ms/step - loss: 1.4312 - accuracy: 0.5013 - val_loss: 1.3933 - val_accuracy: 0.5103
Epoch 5/10
1088/1088 [==============================] - 8s 8ms/step - loss: 1.4136 - accuracy: 0.5078 - val_loss: 1.3789 - val_accuracy: 0.5166
Epoch 6/10
1088/1088 [==============================] - 10s 9ms/step - loss: 1.4015 - accuracy: 0.5134 - val_loss: 1.3692 - val_accuracy: 0.5195
Epoch 7/10
1088/1088 [==============================] - 10s 9ms/step - loss: 1.3926 - accuracy: 0.5173 - val_loss: 1.3600 - val_

In [29]:
with open('drive/MyDrive/Trainable_Embedding_Model.pickle', 'wb') as file:
  pickle.dump(model, file)

In [30]:
y_pred = model.predict(x_train[200:250])

2/2 [==============================] - 3s 11ms/step
[0, 0, 0, 14, 0, 0, 2, 4, 6, 2, 0, 6, 0, 14, 0, 0, 0, 14, 6, 14, 14, 0, 6, 0, 0, 0, 6, 0, 0, 0, 2, 0, 6, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
